# JJ chain calibration

In [18]:
import gdspy
from importlib import reload
from itertools import product

reload(gdspy)

lib = gdspy.GdsLibrary()

#######################################################################################

#create base layer with main ground and markers
main_cell = lib.new_cell('main_cell')



unground = gdspy.Rectangle((-4750,-4750), (4750, 4750), layer=2)
main_cell.add(unground)


delta_cell = 9 / 4

delta_small_cell = 9 / 4 / 3
x_direction = (-3/2, -1/2, 1/2, 3/2)
y_direction = (-3/2, -1/2, 1/2, 3/2)
a = list(product(x_direction, y_direction)) + [(0., 0.)]
d = 200
x_ = 550
y_ = 315
for i in a:
    start_x = (0 + i[0] * delta_cell) *1000 - x_
    start_y = (0 + i[1] * delta_cell) *1000 - y_
    num = gdspy.Text(str(d), 70, (start_x, start_y), layer=71)
    d += 25
    main_cell.add(num)
    

#create markers
id_ = [(-1, 1), (1, 1), (-1, -1), (1, -1)]

length_sq = 1000

for i in id_:
    corner = (i[0]*4750 - i[0]*250, i[1]*4750 - i[1]*250)
    center = (i[0]*4750 + i[0]*250, i[1]*4750 + i[1]*250)
    print(center)
    
    gr =  gdspy.Rectangle((corner[0] + i[0]*length_sq, corner[1]), (corner[0], corner[1] + i[1]*length_sq), layer=2)
    main_cell.add(gr)
    
    for j in id_:
        w = 20
        marker = gdspy.Rectangle((center[0] + j[0]*100 - w, center[1]+ j[1]*100 - w),
                                   (center[0] + j[0]*100 + w, center[1]+ j[1]*100 + w),  layer=3)
        
        main_cell.add(marker)
        
        
#logo
logo = gdspy.Text("Andersen QuTech", 400, (-1800, -5400), layer=2)
main_cell.add(logo)


# add arrows for deposition

arrow1_part1 = gdspy.Rectangle((-3400+700, 5300 - 500), (-3400+700+3700, 5300 + 500), layer=100)
main_cell.add(arrow1_part1)

arrow1_part2 = gdspy.Rectangle((-3400+700+3700+1100, 5300 - 500), (-3400+700+3700+1100+1000, 5300 + 500), layer=100)
main_cell.add(arrow1_part2)


arrow2_part1 = gdspy.Rectangle((5300 - 500, -2000+700), (5300 + 500, 2400), layer=100)
main_cell.add(arrow2_part1)

arrow2_part2 = gdspy.Rectangle((5300 - 500, -3400), (5300 + 500, -2400), layer=100)
main_cell.add(arrow2_part2)

# create orientation arrow
op_arrow_part1 = gdspy.Rectangle((4380 - 30, -3800+250), (4380 + 30, -3800-700+250), layer=70)
op_arrow_part2 = gdspy.Polygon([(4380-60, -3800-700+250), (4380, -3800-700-40+250), (4380+60, -3800-700+250)], layer=70)
main_cell.add(op_arrow_part1)
main_cell.add(op_arrow_part2)


#######################################################################################

##### N SHOULD BE EVEN!!!!!!!!!!
# N is a number of JJ in the chain
N = 100

if int(N / 2) % 2 != 0:
    raise ValueError('N / 2 should be even!!!')

delta_x = 0.5
# delta_x_base = 1.5
delta_x_base = 1.5
# nanowire_length is  the length between nanowire's pads
nanowire_length = 130 + 3
real_nanowire_length = nanowire_length - delta_x * 2 - delta_x_base * 2

# very important shift at x direction
SHIFT = - nanowire_length / 2
nanowire_width = 1.3

delta = 0.200  # initial width of pad


nu_cell = 0


start_x = 0
start_y = 0

stop_x = start_x + nanowire_length
stop_y = start_y

# for vertical and horizontallines
step = 1.2
delta_W = (real_nanowire_length - 2 * step) / (N / 2 - 1)
delta_H = 3
print(delta_W)

corr1 = 1.4
corr2 = 1

u = 1

while delta < 0.610:
    cell = lib.new_cell('jj_chain' + str(nu_cell))
    
#     # add extra metal
#     left_extra_pad = gdspy.Rectangle(
#         (start_x+SHIFT, start_y - nanowire_width / 2),
#         (start_x + delta_x_base+SHIFT, start_y + nanowire_width / 2),
#         layer=53)

#     right_extra_pad = gdspy.Rectangle(
#         (stop_x+SHIFT, stop_y - nanowire_width / 2),
#         (stop_x - delta_x_base+SHIFT, stop_y + nanowire_width / 2),
#         layer=53)

#     cell.add(left_extra_pad)
#     cell.add(right_extra_pad)

    # left extra pad
    point_left_extra_pad = [
        (start_x + delta_x_base - 1.5 + SHIFT, start_y + 0.7 / 2),
        (start_x + delta_x_base + delta_x - 0.2 + SHIFT, start_y + 0.7 / 2),
        (start_x + delta_x_base + delta_x+SHIFT, start_y + delta / 2),
        (start_x + delta_x_base + delta_x+SHIFT, start_y - delta / 2),
        (start_x + delta_x_base + delta_x - 0.2 + SHIFT, start_y - 0.7 / 2),
        (start_x + delta_x_base - 1.5 + SHIFT, start_y - 0.7 / 2)
    ]

    left_extra_pad = gdspy.Polygon(point_left_extra_pad, layer=54)
    cell.add(left_extra_pad)
    
    ban1 = gdspy.Rectangle((start_x + delta_x_base - 1.5 + SHIFT - 0.2, start_y - 0.7 / 2 - 0.2),
                           (start_x + delta_x_base + delta_x + SHIFT + 0.2, start_y + 0.7 / 2 + 0.2),
                          layer=56)
    
    cell.add(ban1)
    
    # left line
    
    left_line = gdspy.Rectangle((start_x + delta_x_base + delta_x+SHIFT, start_y - delta / 2),
                                (start_x + delta_x_base + delta_x + step + delta+corr2+SHIFT, start_y + delta / 2), layer=54)
    cell.add(left_line)
    
    undercut_left_line = gdspy.Rectangle((start_x + delta_x_base + delta_x + step + delta+corr2+SHIFT, start_y - delta / 2),
                                         (start_x + delta_x_base + delta_x + step + delta+corr2+0.7+SHIFT, start_y + delta / 2),
                                         layer=55)
    cell.add(undercut_left_line)
    
    
    # right extra pad
    
    sm_sh = (1.5 - 0.7) / 2 + 0.7
    
    point_right_extra_pad = [
        (stop_x + SHIFT - sm_sh, stop_y - 1),
        (stop_x + SHIFT - sm_sh, stop_y - 1 + delta_x_base+ delta_x - 0.2),
        (stop_x + SHIFT - sm_sh + 0.7 / 2 - delta / 2, stop_y - 1 + delta_x_base+ delta_x - 0.2 + 0.2),
        (stop_x + SHIFT - sm_sh + 0.7 / 2 + delta / 2, stop_y - 1 + delta_x_base+ delta_x - 0.2 + 0.2),
        (stop_x + SHIFT - sm_sh + 0.7, stop_y - 1 + delta_x_base+ delta_x - 0.2),
        (stop_x + SHIFT - sm_sh + 0.7, stop_y - 1),
        
    ]
    
    right_extra_pad = gdspy.Polygon(point_right_extra_pad, layer=54)
    cell.add(right_extra_pad)
    
    ban2 = gdspy.Rectangle((stop_x + SHIFT - sm_sh -0.2, stop_y - 1-0.2),
                           (stop_x + SHIFT - sm_sh + 0.7 +0.2, stop_y - 1 + delta_x_base+ delta_x - 0.2 + 0.2+0.2), 
                          layer=56)
    cell.add(ban2)
    
    
    # right vertical line 
    right_vertical_line = gdspy.Rectangle((stop_x + SHIFT - sm_sh + 0.7 / 2 - delta / 2, stop_y - 1 + delta_x_base+ delta_x - 0.2 + 0.2),
                                          (stop_x + SHIFT - sm_sh + 0.7 / 2 + delta / 2, start_y + 6), layer=54)
    
    cell.add(right_vertical_line)
    
    
    undercut_right_vertical_line  = gdspy.Rectangle(
            (stop_x + SHIFT - sm_sh + 0.7 / 2 - delta / 2,
             start_y + 6),
            (stop_x + SHIFT - sm_sh + 0.7 / 2 + delta / 2,
             start_y + 6 + u),
            
            layer=55)
    
    
    cell.add(undercut_right_vertical_line)

#     # right extra pad
#     point_right_extra_pad = [
#         (stop_x - delta_x_base + 1.5+SHIFT, stop_y + 0.7 / 2),
#         (stop_x - delta_x_base - delta_x + 0.2+SHIFT, stop_y + 0.7 / 2),
#         (stop_x - delta_x_base - delta_x+SHIFT, stop_y + delta / 2),
#         (stop_x - delta_x_base - delta_x+SHIFT, stop_y - delta / 2),
#         (stop_x - delta_x_base - delta_x + 0.2+SHIFT, stop_y - 0.7 / 2),
#         (stop_x - delta_x_base + 1.5+SHIFT, stop_y - 0.7 / 2)
#     ]

#     right_extra_pad = gdspy.Polygon(point_right_extra_pad, layer=54)
#     cell.add(right_extra_pad)
    
    
#     # right line
    
#     right_line = gdspy.Rectangle((stop_x - delta_x_base - delta_x+SHIFT, stop_y + delta / 2),
#                                 (stop_x - delta_x_base - delta_x - step - corr2+SHIFT, stop_y - delta / 2), layer=54)
#     cell.add(right_line)
    

    
    chain_JJ = None
    k = 0 #  center
    p = 0 #  down 
    for i in range(int(N / 2)):
        
        

        vertical_pad = gdspy.Rectangle(
            (start_x + delta_x_base + delta_x + step + delta_W * i+SHIFT,
             start_y - 6),
            (start_x + delta_x_base + delta_x + step + delta_W * i + delta+SHIFT,
             start_y + 6),
            layer=54)
        
        
        undercut_vertical = gdspy.Rectangle(
            (start_x + delta_x_base + delta_x + step + delta_W * i+SHIFT,
             start_y + 6),
            (start_x + delta_x_base + delta_x + step + delta_W * i + delta+SHIFT,
             start_y + 6 + u),
            
            layer=55)

        chain_JJ = gdspy.boolean(chain_JJ, vertical_pad, 'or')

        cell.add(vertical_pad)
        cell.add(undercut_vertical)

        if i % 2 != 0: #and i != int(N / 2) - 1:
            
            if k % 2 == 0:
                horizonatal_center = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W * i - corr1+SHIFT,
                 start_y - delta / 2+ 3),
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2+SHIFT, start_y + delta / 2+ 3),
                layer=54)
                
                undercut_horizontal_center = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2+SHIFT, start_y - delta / 2+ 3),
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2 + u+SHIFT, start_y + delta / 2+ 3),
                layer=55)
                
            else:
                horizonatal_center = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W * i - corr1+SHIFT,
                 start_y - delta / 2+1),
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2+SHIFT, start_y + delta / 2+1),
                layer=54)
                
                undercut_horizontal_center = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2+SHIFT, start_y - delta / 2 +1),
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2 + u+SHIFT, start_y + delta / 2+1),
                layer=55)
                
                
            k +=1
        
                

            

            chain_JJ = gdspy.boolean(chain_JJ, horizonatal_center, 'or')

            cell.add(horizonatal_center)

            

            cell.add(undercut_horizontal_center)

        if i % 2 == 0:
            
            if p % 2 == 0:
                
                horizonatal_down = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W * i - corr1+SHIFT,
                 start_y - delta_H - delta),
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2+SHIFT, start_y - delta_H),
                layer=54)
                
                undercut_horizontal_center = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2+SHIFT, start_y - delta_H),
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2 +u+SHIFT, start_y - delta_H - delta),
                layer=55)
                
            else:
                horizonatal_down = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W * i - corr1+SHIFT,
                 start_y - delta_H - delta + 1.5),
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2+SHIFT, start_y - delta_H + 1.5),
                layer=54)
                
                undercut_horizontal_center = gdspy.Rectangle(
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2+SHIFT, start_y - delta_H+1.5),
                (start_x + delta_x_base + delta_x + step + delta_W *
                 (i + 1) + corr2 +u+SHIFT, start_y - delta_H - delta+1.5),
                layer=55)
                
            p += 1

            

            chain_JJ = gdspy.boolean(chain_JJ, horizonatal_down, 'or')

            cell.add(horizonatal_down)

            

            cell.add(undercut_horizontal_center)

    # cell.add(chain_JJ)

    delta += 0.025
    nu_cell += 1

(-5000, 5000)
(5000, 5000)
(-5000, -5000)
(5000, -5000)
2.583673469387755


In [2]:
gdspy.LayoutViewer()

C:\Users\figenyilmaz\AppData\Local\Temp\ipykernel_9712\1312026100.py:1: DeprecationWarning: [GDSPY] Use of the global library is deprecated.  Pass LayoutViewer a GdsLibrary instance.
  gdspy.LayoutViewer()


<gdspy.viewer.LayoutViewer object .!layoutviewer>

In [3]:
lib.write_gds('JJ_chain_150mu_m_101_jj_new.gds')

# Create file with calibration structures (JJ) 

In [19]:
import gdspy
from importlib import reload
reload(gdspy)

delta = 0.04 # initial width of pad
pad_length_left = 2.4
pad_length_right = 1.7

nu_cell = 0
lib = gdspy.GdsLibrary()

while delta < 0.210:
    cell = lib.new_cell('junction_' + str(nu_cell))

    left_big = gdspy.Rectangle((-15., -1.5), (-2., 1.5), layer=53)
    right_big = gdspy.Rectangle((2., -1.5), (15., 1.5), layer=53)
    

    left_small = gdspy.Rectangle((-2., 0.5), (-0.2, 1.5), layer=53)
    right_small = gdspy.Rectangle((0.2, -1.5), (2., -0.5), layer=53)
    
    # Pads of junctions
    left_pad = gdspy.Rectangle((-0.6, 1 - pad_length_left + 0.2), (-0.6 + delta, 1.5), layer=54)
    
    right_pad = gdspy.Rectangle((0.7 - pad_length_right + 0.2, -1.0), (0.7, -1.0 + delta), layer=54)
    
    # parts for undercuts
    undercut_left_pad = gdspy.Rectangle((-0.6, 1 - pad_length_left), (-0.6 + delta, 1 - pad_length_left + 0.2), layer=55)
    
    undercut_right_pad = gdspy.Rectangle((0.7 - pad_length_right, -1.0), (0.7 - pad_length_right + 0.2, -1.0 + delta), layer=55)

    cell.add(left_big)
    cell.add(right_big)

    cell.add(left_small)
    cell.add(right_small)

    cell.add(left_pad)
    cell.add(right_pad)
    
    cell.add(undercut_left_pad)
    cell.add(undercut_right_pad)
    
    nu_cell += 1
    delta += 0.01

In [20]:
gdspy.LayoutViewer()

  gdspy.LayoutViewer()



<gdspy.viewer.LayoutViewer object .!layoutviewer>

In [21]:
lib.write_gds('fluxonium_junctions.gds')

# Create file with calibration structure with gradual narrowing

In [22]:
import gdspy
from importlib import reload

from itertools import product
reload(gdspy)

delta = 0.04 # initial width of pad
pad_length_left = 2.8
pad_length_right = 2.2+0.4

nu_cell = 0
lib = gdspy.GdsLibrary()

#create base layer with main ground and markers
main_cell = lib.new_cell('main_cell')



unground = gdspy.Rectangle((-4750,-4750), (4750, 4750), layer=2)
main_cell.add(unground)
#######################################################################################

delta_cell = 9 / 4

delta_small_cell = 9 / 4 / 3
x_direction = (-3/2, -1/2, 1/2, 3/2)
y_direction = (-3/2, -1/2, 1/2, 3/2)
a = list(product(x_direction, y_direction)) + [(0., 0.)]
d = 40
x_ = 550
y_ = 315
for i in a:
    start_x = (0 + i[0] * delta_cell) *1000 - x_
    start_y = (0 + i[1] * delta_cell) *1000 - y_
    num = gdspy.Text(str(d), 70, (start_x, start_y), layer=71)
    d += 10
    main_cell.add(num)
    

#create markers
id_ = [(-1, 1), (1, 1), (-1, -1), (1, -1)]

length_sq = 1000

for i in id_:
    corner = (i[0]*4750 - i[0]*250, i[1]*4750 - i[1]*250)
    center = (i[0]*4750 + i[0]*250, i[1]*4750 + i[1]*250)
    print(center)
    
    gr =  gdspy.Rectangle((corner[0] + i[0]*length_sq, corner[1]), (corner[0], corner[1] + i[1]*length_sq), layer=2)
    main_cell.add(gr)
    
    for j in id_:
        w = 20
        marker = gdspy.Rectangle((center[0] + j[0]*100 - w, center[1]+ j[1]*100 - w),
                                   (center[0] + j[0]*100 + w, center[1]+ j[1]*100 + w),  layer=3)
        
        main_cell.add(marker)
        
        
#logo
logo = gdspy.Text("Andersen QuTech", 400, (-1800, -5400), layer=2)
main_cell.add(logo)


# add arrows for deposition

arrow1_part1 = gdspy.Rectangle((-3400+700, 5300 - 500), (-3400+700+3700, 5300 + 500), layer=100)
main_cell.add(arrow1_part1)

arrow1_part2 = gdspy.Rectangle((-3400+700+3700+1100, 5300 - 500), (-3400+700+3700+1100+1000, 5300 + 500), layer=100)
main_cell.add(arrow1_part2)


arrow2_part1 = gdspy.Rectangle((5300 - 500, -2000+700), (5300 + 500, 2400), layer=100)
main_cell.add(arrow2_part1)

arrow2_part2 = gdspy.Rectangle((5300 - 500, -3400), (5300 + 500, -2400), layer=100)
main_cell.add(arrow2_part2)

# create orientation arrow
op_arrow_part1 = gdspy.Rectangle((4380 - 30, -3800+250), (4380 + 30, -3800-700+250), layer=70)
op_arrow_part2 = gdspy.Polygon([(4380-60, -3800-700+250), (4380, -3800-700-40+250), (4380+60, -3800-700+250)], layer=70)
main_cell.add(op_arrow_part1)
main_cell.add(op_arrow_part2)

    
############# NEW JUNCTIONS ##################################################################################################  

while delta < 0.210:
    cell = lib.new_cell('junction_' + str(nu_cell))
    
#     left_big_points = [(-15., -1.5), (-10, -1.5), (-2., 0.5), (-2., 1.5), (-15, 1.5)]
#     right_big_points = [(2., -1.5), (15, -1.5), (15., 1.5), (10, 1.5), (2., -0.5)]
    
    left_big_points = [(-15., 0), (-10, 0), (-2., 1.5), (-2., 3), (-15, 3)]
    right_big_points = [(2., -1.5), (15, -1.5), (15., 1.5), (10, 1.5), (2., 0)]

    left_big = gdspy.Polygon(left_big_points, layer=53)
    right_big = gdspy.Polygon(right_big_points, layer=53)
    
#     left_small = gdspy.Rectangle((-2., 0.5), (-0.2, 1.5), layer=53)
    
#     right_small = gdspy.Rectangle((0.2, -1.5), (2., -0.5), layer=53)

    left_small = gdspy.Rectangle((-2., 1.5), (-0.2, 3), layer=53)
    
    right_small = gdspy.Rectangle((0.2, -1.5), (2., 0), layer=53)
    
    
    
    # Pads of junctions
    left_pad = gdspy.Rectangle((-1.1 - 0.4 - delta / 2, 0.75 - pad_length_left + 0.2), (-1.1 - 0.4 + delta/2, 1), layer=54)
    
    right_pad = gdspy.Rectangle((-0.2 - pad_length_right + 0.2, -0.75-delta/2), (-0.2, -0.75 + delta/2), layer=54)
    
    
    
    left_extra_points = [(-1.1 - 0.4 - delta / 2, 1), (-1.5 -0.4 , 1.2) ,  (-1.5-0.4, 3.5),
                         (-0.7-0.4, 3.5),  (-0.7-0.4 , 1.2),  (-1.1 -0.4  + delta / 2, 1)]
    
    right_extra_points = [(-0.2, -0.75 + delta / 2), (0., -0.4),  (2.5, -0.4),
                          (2.5, -1.1), (0., -1.1), (-0.2, -0.75 - delta / 2)]
    
    
    left_pad_extra = gdspy.Polygon(left_extra_points, layer=54)
    right_pad_extra = gdspy.Polygon(right_extra_points, layer=54)
    
    
    #bandages
    ban1 = gdspy.Rectangle((-1.5 -0.4 -0.2, 1.2-0.35),  (-0.7-0.4+0.2, 3.5+0.2), layer=56)
    ban2 = gdspy.Rectangle((0. - 0.35, -0.4+0.2),  (2.5+0.2, -1.1-0.2), layer=56)
    
    
#     left_pad_extra = gdspy.Rectangle((-1.5, 1), (-0.7, 3.5), layer=54)
    
#     right_pad_extra = gdspy.Rectangle((-0.2, 0-0.4), (1.5, -1.5+0.4), layer=54)
   
    

    # parts for undercuts
    undercut_left_pad = gdspy.Rectangle((-1.1 - 0.4- delta / 2, 0.75 - pad_length_left + 0.2 - 0.55),
                                        (-1.1 -0.4  + delta/2, 0.75 - pad_length_left + 0.2), layer=55)
    
    undercut_right_pad = gdspy.Rectangle((-0.2 - pad_length_right + 0.2-0.55, -0.75-delta/2),
                                         (-0.2 - pad_length_right + 0.2,-0.75+delta/2), layer=55)
    
   

    cell.add(left_big)
    cell.add(right_big)

    cell.add(left_small)
    cell.add(right_small)

    cell.add(left_pad)
    cell.add(right_pad)
    
    cell.add(left_pad_extra)
    cell.add(right_pad_extra)
    
    
    cell.add(ban1)
    
    cell.add(ban2)
    
    cell.add(undercut_left_pad)
    cell.add(undercut_right_pad)
    
    nu_cell += 1
    delta += 0.01
############# OLD JUNCTIONS ##################################################################################################   
nu_cell=0 
delta = 0.04 # initial width of pad
while delta < 0.210:
    cell = lib.new_cell('junction_old' + str(nu_cell))
    
    left_big_points = [(-15., 0), (-10, 0), (-2., 1.5), (-2., 3), (-15, 3)]
    right_big_points = [(2., -1.5), (15, -1.5), (15., 1.5), (10, 1.5), (2., 0)]

    left_big = gdspy.Polygon(left_big_points, layer=53)
    right_big = gdspy.Polygon(right_big_points, layer=53)

    left_small = gdspy.Rectangle((-2., 1.5), (-0.2, 3), layer=53)
    
    right_small = gdspy.Rectangle((0.2, -1.5), (2., 0), layer=53)
    
    #bandages
    ban1 = gdspy.Rectangle((-1.5 -0.4 -0.2, 1.2-0.35),  (-0.7-0.4+0.2, 3.5+0.2), layer=56)
    ban2 = gdspy.Rectangle((0. - 0.35, -0.4+0.2),  (2.5+0.2, -1.1-0.2), layer=56)
    
    
    
    # Pads of junctions
    
    left_pad = gdspy.Rectangle((-1.1 - 0.4 - delta / 2, 0.75 - pad_length_left + 0.2), (-1.1 - 0.4  + delta/2, 2.5+0.5), layer=54)
    
    right_pad = gdspy.Rectangle((-0.2 - pad_length_right + 0.2, -0.75-delta/2), (1.5+0.5, -0.75 + delta/2), layer=54)
    
 
    # parts for undercuts
    undercut_left_pad = gdspy.Rectangle((-1.1 - 0.4 - delta / 2, 0.75 - pad_length_left + 0.2 - 0.55),
                                        (-1.1 - 0.4 + delta/2, 0.75 - pad_length_left + 0.2), layer=55)
    
    undercut_right_pad = gdspy.Rectangle((-0.2 - pad_length_right + 0.2-0.55, -0.75-delta/2),
                                         (-0.2 - pad_length_right + 0.2,-0.75+delta/2), layer=55)
    
    
   

    cell.add(left_big)
    cell.add(right_big)

    cell.add(left_small)
    cell.add(right_small)

    cell.add(left_pad)
    cell.add(right_pad)
    
    cell.add(ban1)
    cell.add(ban2)
    
    cell.add(undercut_left_pad)
    cell.add(undercut_right_pad)
    
    nu_cell += 1
    delta += 0.01
    
    

(-5000, 5000)
(5000, 5000)
(-5000, -5000)
(5000, -5000)


In [23]:
gdspy.LayoutViewer()

  gdspy.LayoutViewer()



<gdspy.viewer.LayoutViewer object .!layoutviewer>

In [24]:
lib.write_gds('fluxonium_junctions_gradual.gds')

# 

# Create geometry of a chip

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from collections import OrderedDict

import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, open_docs
import matplotlib.pyplot as plt

from qiskit_metal.toolbox_metal import math_and_overrides

from qiskit_metal.qlibrary.core import QComponent

from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.pathfinder import RoutePathfinder
from qiskit_metal.qlibrary.tlines.anchored_path import RouteAnchors

from qiskit_metal.qlibrary.qubits.fluxoniumEPR import FluxoniumPocket

from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond

# from qiskit_metal.qlibrary.terminations.open_to_ground_v2 import OpenToGround
from itertools import product

In [2]:
design = designs.DesignPlanar()
design.chips.main.material = 'silicon'
design.variables['cpw_width'] = '15 um'
design.variables['cpw_gap'] = '8.733 um'
design._chips['main']['size']['size_x'] = '9mm'
design._chips['main']['size']['size_y'] = '9mm'
design.overwrite_enabled = True

gui = MetalGUI(design)

In [11]:
# cell_start_x = str(0 + 2.5 + 2.5 / 2) + 'mm'
# cell_start_y = str(0 - 2.5 + 2.5 / 2) + 'mm'

# delta_cell_x = '2.5mm'
# delta_cell_y = '2.5mm'



In [3]:
delta_cell = 9 / 4

delta_small_cell = 9 / 4 / 3
x_direction = (-3/2, -1/2, 1/2, 3/2)
y_direction = (-3/2, -1/2, 1/2, 3/2)
a = list(product(x_direction, y_direction)) + [(0., 0.)]
a

[(-1.5, -1.5),
 (-1.5, -0.5),
 (-1.5, 0.5),
 (-1.5, 1.5),
 (-0.5, -1.5),
 (-0.5, -0.5),
 (-0.5, 0.5),
 (-0.5, 1.5),
 (0.5, -1.5),
 (0.5, -0.5),
 (0.5, 0.5),
 (0.5, 1.5),
 (1.5, -1.5),
 (1.5, -0.5),
 (1.5, 0.5),
 (1.5, 1.5),
 (0.0, 0.0)]

In [4]:
for i in a:
    x_ = 700
    y_ = 485
    cell_start_x = (0 + i[0] * delta_cell) *1000 - x_
    cell_start_y = (0 + i[1] * delta_cell) *1000 - y_
    print(cell_start_x, cell_start_y)

-4075.0 -3860.0
-4075.0 -1610.0
-4075.0 640.0
-4075.0 2890.0
-1825.0 -3860.0
-1825.0 -1610.0
-1825.0 640.0
-1825.0 2890.0
425.0 -3860.0
425.0 -1610.0
425.0 640.0
425.0 2890.0
2675.0 -3860.0
2675.0 -1610.0
2675.0 640.0
2675.0 2890.0
-700.0 -485.0


In [5]:
from qiskit_metal.qlibrary.qubits.fluxoniumEPR import FluxoniumPocket
# from itertools import product
# x_direction = (-3/2, -1/2, 1/2, 3/2)
# y_direction = (-3/2, -1/2, 1/2, 3/2)
nu_qubit = 0
nu_cell = 0
all_qubits = []
all_labels = []
# cell_name = 'junction_'
cell_name = 'jj_chain'
for i in a:
# for i in [(0, 0)]:
    all_labels.append(i)
    print('LABEL:', i)
    gds_cell_name_ = cell_name + str(nu_cell)
    for j in [-1, 0, 1]:
        
#         if j == 1:
#             gds_cell_name_ = cell_name + 'old'+ str(nu_cell)
#         else:
#             gds_cell_name_ = cell_name + str(nu_cell)
    
        cell_start_x = str(0 + i[0] * delta_cell + j * (delta_small_cell - 0.5)) + 'mm'
        cell_start_y = str(0 + i[1] * delta_cell) + 'mm'

        pos_x_ = cell_start_x
        pos_y_ = cell_start_y

        position = (pos_x_, pos_y_)
        print('Qubit-position:', position)

        options = dict(chip='main',
                       orientation=0, pos_x=pos_x_, pos_y=pos_y_, nanowire = True,
                      
                       gds_cell_name=gds_cell_name_,

                       flux_bias_line_options=Dict(make_fbl=False,
                                                   fbl_sep='100um',
                                                   ),
                       charge_line_options=Dict(loc_H=-1, make_cl=False,
                                                cl_sep='-10um',
                                                ),
                       readout_line_options=Dict(loc_H=+1, make_rol=False,
                                                 pad_width='300um', pad_height='80um',
                                                 pad_sep='60um',
                                                 ))
        
        print(gds_cell_name_)

        q = FluxoniumPocket(design, 'Q' + str(nu_qubit), options=dict(
            **options))
        
        all_qubits.append(q)

        gui.rebuild()
        nu_qubit += 1
    nu_cell += 1

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  vertices = concatenate([

  vertices = concatenate([

  vertices = concatenate([



LABEL: (-1.5, -1.5)
Qubit-position: ('-3.625mm', '-3.375mm')
jj_chain0


  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

 

Qubit-position: ('-3.375mm', '-3.375mm')
jj_chain0


  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  vertices = concatenate([

  vertices = concatenate([

  vertices = concatenate([

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = ta

...

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

 

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

 

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

  self.tables[kind] = table.append(df, sort=False, ignore_index=True)

  super().__setattr__(name, value)

 

In [9]:
gui.main_window.close()

True

# GDS

In [6]:
fluxonium_gds = design.renderers.gds
fluxonium_gds.options

{'short_segments_to_not_fillet': 'True',
 'check_short_segments_by_scaling_fillet': '2.0',
 'gds_unit': 0.001,
 'ground_plane': 'True',
 'negative_mask': {'main': []},
 'fabricate': 'False',
 'corners': 'circular bend',
 'tolerance': '0.00001',
 'precision': '0.000000001',
 'width_LineString': '10um',
 'path_filename': '../resources/Fake_Junctions.GDS',
 'junction_pad_overlap': '5um',
 'max_points': '199',
 'cheese': {'datatype': '100',
  'shape': '0',
  'cheese_0_x': '25um',
  'cheese_0_y': '25um',
  'cheese_1_radius': '100um',
  'view_in_file': {'main': {1: True}},
  'delta_x': '100um',
  'delta_y': '100um',
  'edge_nocheese': '200um'},
 'no_cheese': {'datatype': '99',
  'buffer': '25um',
  'cap_style': '2',
  'join_style': '2',
  'view_in_file': {'main': {1: True}}},
 'bounding_box_scale_x': '1.2',
 'bounding_box_scale_y': '1.2'}

In [10]:
# Choose file (with gradual junctions or note)
# fluxonium_gds.options['path_filename'] = 'fluxonium_junctions.gds'
# fluxonium_gds.options['path_filename'] = 'fluxonium_junctions_gradual.gds'
fluxonium_gds.options['path_filename'] = 'JJ_chain_150mu_m_101_jj_new.gds'

In [11]:
fluxonium_gds.options

{'short_segments_to_not_fillet': 'True',
 'check_short_segments_by_scaling_fillet': '2.0',
 'gds_unit': 0.001,
 'ground_plane': 'True',
 'negative_mask': {'main': []},
 'fabricate': 'False',
 'corners': 'circular bend',
 'tolerance': '0.00001',
 'precision': '0.000000001',
 'width_LineString': '10um',
 'path_filename': 'JJ_chain_150mu_m_101_jj_new.gds',
 'junction_pad_overlap': '5um',
 'max_points': '199',
 'cheese': {'datatype': '100',
  'shape': '0',
  'cheese_0_x': '25um',
  'cheese_0_y': '25um',
  'cheese_1_radius': '100um',
  'view_in_file': {'main': {1: True}},
  'delta_x': '100um',
  'delta_y': '100um',
  'edge_nocheese': '200um'},
 'no_cheese': {'datatype': '99',
  'buffer': '25um',
  'cap_style': '2',
  'join_style': '2',
  'view_in_file': {'main': {1: True}}},
 'bounding_box_scale_x': '1.2',
 'bounding_box_scale_y': '1.2'}

In [12]:
# fluxonium_gds.export_to_gds('fluxonium_calibration_structure.gds')
fluxonium_gds.export_to_gds('fluxonium_calibration_structure_Martinis_idea_JJ_chain-3.gds')

02:26PM 58s WARNING [_give_rotation_center_twopads]: In junction table, component=Q0 with name=inductor has width=0.001 smaller than cell dimension=0.013.
02:26PM 58s WARNING [_give_rotation_center_twopads]: In junction table, component=Q0 with name=rect_jj has width=0.01 smaller than cell dimension=0.013.


ValueError: [GDSPY] Cell named pads_jj_chain0_QComponent_is_1 already present in library.

In [17]:
fluxonium_gds.options['fabricate'] = True

In [18]:
fluxonium_gds.export_to_gds('fluxonium_calibration_structure_Martinis_idea_JJ_chain_fab.gds')

03:10PM 11s WARNING [_import_junctions_to_one_cell]: Not able to find file:"JJ_chain_150mu_m_101_jj_newFGN.gds".  Not used to replace junction. Checked directory:"C:\Users\figenyilmaz\Desktop\qiskit-metal\tutorials_ME\Figen\Fluxonuim\Calibraitions_GDS".
  all_polys = list(multi_poly)

  all_polys = list(multi_poly)



1